In [1]:
import pandas as pd
import technical.indicators as ftt
from functools import reduce
from datetime import datetime, timedelta
import numpy as np
import plotly.graph_objects as go
import re
import os
import talib.abstract as ta

In [2]:
files = os.listdir("./data_research")

In [3]:
ada_usdt_files = [file_name for file_name in os.listdir("./data_research") if bool(re.match("ADAUSDT", file_name))]

In [4]:
ada_usdt = []
for file in ada_usdt_files:
    df = pd.read_csv(
        f"./data_research/{file}",
        names=[
            'open_time',
            'open',
            'high',
            'low',
            'close',
            'volume',
            'close_time',
            'quote_volume',
            'count',
            'taker_buy_volume',
            'taker_buy_quote_volume',
            'ignore'
        ]
    )
    ada_usdt.append(df)
    
ada_usdt = pd.concat(ada_usdt, axis=0, ignore_index=True)

In [5]:
ada_usdt.drop(['open_time', 'quote_volume', 'count', 'taker_buy_volume', 'taker_buy_quote_volume', 'ignore'], axis=1, inplace=True)

In [6]:
ada_usdt.rename(columns={'close_time': 'datetime'}, inplace=True)

In [7]:
ada_usdt.sort_values(by='datetime', inplace=True)

In [8]:
ada_usdt.set_index('datetime', inplace=True)

In [9]:
ada_usdt.index = pd.to_datetime(ada_usdt.index, unit='ms')

In [10]:
ada_usdt.head()

,open,high,low,close,volume
datetime,,,,,
2023-09-01 00:04:59.999,0.2554,0.2556,0.2550,0.2554,847128.5
2023-09-01 00:09:59.999,0.2555,0.2556,0.2553,0.2555,362375.9
2023-09-01 00:14:59.999,0.2555,0.2556,0.2555,0.2556,176382.6
2023-09-01 00:19:59.999,0.2556,0.2581,0.2556,0.2570,5336822.6
2023-09-01 00:24:59.999,0.2570,0.2572,0.2564,0.2565,333397.7


# Populate Indicators

In [ ]:
length1_buy = 13
length2_buy = 19
length3_buy = 21
length4_buy = 39
length5_buy = 50
length6_buy = 200


In [ ]:
def populate_indicators(
        dataframe: pd.DataFrame,
        length1:int=13,
        length2:int=19,
        length3:int=21,
        length4:int=39,
        length5:int=50,
        length6:int=200
) -> pd.DataFrame:
    # The source for calculation is the close price    
    source = 'close'
    
    # Basis line is the length6 (SMA200) of close price
    dataframe['basis'] = ta.SMA(dataframe[source], timeperiod = length6)
    
    # Dev is the length6 (200)-period rolling volatility (stdev) of the close price
    dataframe['dev'] = dataframe[source].rolling(length6).std()
    
    # Upper and lower band is the SMA200 +- volatility
    dataframe['upperBand'] = dataframe['basis'] + dataframe['dev']
    dataframe['lowerBand'] = dataframe['basis'] - dataframe['dev']
    
    
    dataframe['bPct'] = np.where(
        (dataframe['upperBand'] - dataframe['lowerBand'] ) != 0, 
        (dataframe[source] - dataframe['lowerBand'])/(dataframe['upperBand'] - dataframe['lowerBand']),
        0
    )
    
    # advances sum detects where the first difference of close price is positive, then takes the sum of the 
    # length2-rolling window
    dataframe['advSum'] = pd.Series(np.where(dataframe[source].diff() > 0, 1, 0)).rolling(length2).sum()
    
    # declines sum detects where the first difference of close price is non-positive, then takes the sum of the 
    # length2-rolling window
    dataframe['decSum'] = pd.Series(np.where(dataframe[source].diff() > 0, 0, 1)).rolling(length2).sum()
    
    # ratio of advSum and decSum
    dataframe['ratio'] = np.where(
        dataframe['decSum'] != 0, 
        dataframe['advSum'] / dataframe['decSum'],
        0
    )
    
    # Ratio-Adjusted Net Advances (RANA): (advances - declines) / (advances + declines)
    dataframe['rana'] = np.where(
        (dataframe['advSum'] + dataframe['decSum']) != 0,
        (dataframe['advSum'] - dataframe['decSum'])/(dataframe['advSum'] + dataframe['decSum']), 
        0
    )
    
    # McClellan Oscillator: 19-day EMA of RANA - 39-day EMA of RANA
    dataframe['mo'] = ta.EMA(dataframe['rana'], timeperiod = length2) - ta.EMA(dataframe['rana'], timeperiod = length4) 
    
    
    dataframe['utm'] = \
        (200 * dataframe['bPct']) + \
        (100 * dataframe['ratio']) + \
        (2 * dataframe['mo']) + \
        (1.5 * ta.MFI(dataframe, timeperiod = length5) ) + \
        (3 * ta.MFI(dataframe, timeperiod = length3) ) + \
        (3 * ta.MFI(dataframe, timeperiod = length1) ) 
    dataframe['utmiRsi']  = ta.RSI(dataframe['utm'], timeperiod = length1)
    dataframe['utmi'] = ta.EMA(dataframe['utmiRsi'], timeperiod = length1)



    return dataframe